In [ ]:
!pip install openai python-dotenv


# Set Environment Variables

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")


In [ ]:
import os

os.environ["MODEL_NAME"] = "gpt-4o-mini"
os.environ["TEMPERATURE"] = "0.3"
os.environ["MAX_TOKENS"] = "800"


In [ ]:
import os

print("API key loaded:", os.environ.get("OPENAI_API_KEY") is not None)
print("Model:", os.environ.get("MODEL_NAME"))


API key loaded: True
Model: gpt-4o-mini


In [ ]:
%%writefile config.py
import os

MODEL_NAME = os.getenv("MODEL_NAME")
TEMPERATURE = float(os.getenv("TEMPERATURE"))
MAX_TOKENS = int(os.getenv("MAX_TOKENS"))


Overwriting config.py


# Create Provider Layer
Base Provider

In [ ]:
%%writefile base_provider.py
from abc import ABC, abstractmethod

class BaseProvider(ABC):

    @abstractmethod
    def generate(self, prompt: str):
        pass


Overwriting base_provider.py


# OpenAI Provider

In [ ]:
%%writefile openai_provider.py
from openai import OpenAI
from config import MODEL_NAME, TEMPERATURE, MAX_TOKENS

class OpenAIProvider:
    def __init__(self):
        self.client = OpenAI()

    def generate(self, prompt: str):
        response = self.client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS
        )
        return response.choices[0].message.content


Overwriting openai_provider.py


# Prompt Functions
Flashcard Prompt

In [ ]:
%%writefile flashcard_prompt.py
def flashcard_prompt(text):
    return f"""
Generate flashcards.

Return JSON only.

Format:
[
  {{
    "question": "...",
    "answer": "..."
  }}
]

Text:
{text}
"""


Overwriting flashcard_prompt.py


# Quiz Prompt

In [ ]:
%%writefile quiz_prompt.py
def quiz_prompt(text):
    return f"""
Generate MCQ questions.

Return JSON only.

Format:
[
  {{
    "question": "...",
    "options": ["A","B","C","D"],
    "correct_answer": "A"
  }}
]

Text:
{text}
"""


Overwriting quiz_prompt.py


# Summary Prompt

In [ ]:
%%writefile summary_prompt.py
def summary_prompt(text):
    return f"""
Summarize the text clearly.

Text:
{text}
"""


Overwriting summary_prompt.py


# Evaluation Prompt

In [ ]:
%%writefile evaluation_prompt.py
def evaluation_prompt(correct, user):
    return f"""
Evaluate the answer.

Correct:
{correct}

User:
{user}

Return JSON:
{{
  "score": 0-100,
  "feedback": "short feedback"
}}
"""


Overwriting evaluation_prompt.py


# AI Engines
Flashcard Engine

In [ ]:
%%writefile flashcard_engine.py
from flashcard_prompt import flashcard_prompt

class FlashcardEngine:
    def __init__(self, provider):
        self.provider = provider

    def generate(self, text):
        return self.provider.generate(flashcard_prompt(text))


Overwriting flashcard_engine.py


# Quiz Engine

In [ ]:
%%writefile quiz_engine.py
from quiz_prompt import quiz_prompt

class QuizEngine:
    def __init__(self, provider):
        self.provider = provider

    def generate(self, text):
        return self.provider.generate(quiz_prompt(text))


Overwriting quiz_engine.py


# Summary Engine

In [ ]:
%%writefile summary_engine.py
from summary_prompt import summary_prompt

class SummaryEngine:
    def __init__(self, provider):
        self.provider = provider

    def generate(self, text):
        return self.provider.generate(summary_prompt(text))


Overwriting summary_engine.py


# Evaluation Engine

In [ ]:
%%writefile evaluation_engine.py
from evaluation_prompt import evaluation_prompt

class EvaluationEngine:
    def __init__(self, provider):
        self.provider = provider

    def evaluate(self, correct, user):
        return self.provider.generate(evaluation_prompt(correct, user))


Overwriting evaluation_engine.py


# Test Everything MAIN CELL

In [51]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from openai_provider import OpenAIProvider
from flashcard_engine import FlashcardEngine
from quiz_engine import QuizEngine
from summary_engine import SummaryEngine
from evaluation_engine import EvaluationEngine

provider = OpenAIProvider()

text = """
Machine learning is a subset of artificial intelligence
that enables systems to learn from data.
"""

print("FLASHCARDS:")
print(FlashcardEngine(provider).generate(text))

print("\nQUIZ:")
print(QuizEngine(provider).generate(text))

print("\nSUMMARY:")
print(SummaryEngine(provider).generate(text))

print("\nEVALUATION:")
print(EvaluationEngine(provider).evaluate(
    "ML allows systems to learn from data",
    "ML learns automatically"
))

FLASHCARDS:
[
  {
    "question": "What is machine learning?",
    "answer": "A subset of artificial intelligence that enables systems to learn from data."
  }
]

QUIZ:
[
  {
    "question": "What is machine learning a subset of?",
    "options": ["Artificial Intelligence", "Data Science", "Statistics", "Computer Programming"],
    "correct_answer": "A"
  },
  {
    "question": "What does machine learning enable systems to do?",
    "options": ["Learn from data", "Perform calculations", "Store information", "Execute commands"],
    "correct_answer": "A"
  },
  {
    "question": "Which of the following best describes machine learning?",
    "options": ["A method for data analysis", "A type of programming language", "A hardware component", "A database management system"],
    "correct_answer": "A"
  },
  {
    "question": "Is machine learning considered a part of artificial intelligence?",
    "options": ["Yes", "No", "Only in certain cases", "Not applicable"],
    "correct_answer": "A"
